In [75]:
import pandas as pd
import glob
import os
#Loading and combining all the data
path ="./Dataofprevious12months"
allDataFiles = glob.glob(os.path.join(path, "*.csv"))
#Merging all the data to one dataframe
df= pd.concat((pd.read_csv(x) for x in allDataFiles ), ignore_index=True)


df["end_station_name"].isna().mean() * 100


df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"])


df.isnull().sum()

df["ride_length"] = (
    df["ended_at"] - df["started_at"]
).dt.total_seconds() 


df["ride_length"]
df.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length
0,7569BC890583FCD7,classic_bike,2025-01-21 17:23:54.538,2025-01-21 17:37:52.015,Wacker Dr & Washington St,KA1503000072,McClurg Ct & Ohio St,TA1306000029,41.883143,-87.637242,41.892592,-87.617289,member,837.477
1,013609308856B7FC,electric_bike,2025-01-11 15:44:06.795,2025-01-11 15:49:11.139,Halsted St & Wrightwood Ave,TA1309000061,Racine Ave & Belmont Ave,TA1308000019,41.929147,-87.649153,41.939743,-87.658865,member,304.344
2,EACACD3CE0607C0D,classic_bike,2025-01-02 15:16:27.730,2025-01-02 15:28:03.230,Southport Ave & Waveland Ave,13235,Broadway & Cornelia Ave,13278,41.948226,-87.664071,41.945529,-87.646439,member,695.500
3,EAA2485BA64710D3,classic_bike,2025-01-23 08:49:05.814,2025-01-23 08:52:40.047,Southport Ave & Waveland Ave,13235,Southport Ave & Roscoe St,13071,41.948226,-87.664071,41.943739,-87.664020,member,214.233
4,7F8BE2471C7F746B,electric_bike,2025-01-16 08:38:32.338,2025-01-16 08:41:06.767,Southport Ave & Waveland Ave,13235,Southport Ave & Roscoe St,13071,41.948226,-87.664071,41.943739,-87.664020,member,154.429


In [ ]:
df['ride_length'] = (df['ended_at'] - df['started_at']).dt.total_seconds()


df['day_of_week'] = df['started_at'].dt.dayofweek
df['day_of_week'] = df['day_of_week'].apply(lambda x: x + 2 if x < 6 else 1)
df['day_of_week']


df_cleaned = df[df['ride_length'] > 0]

df_cleaned.info()
df_cleaned.describe()
df_cleaned['day_of_week'].describe()
df_cleaned['ride_length'].describe()



<class 'pandas.core.frame.DataFrame'>
Index: 5552965 entries, 0 to 5552993
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  ride_length         float64       
 14  day_of_week         int64         
dtypes: datetime64[ns](2), float64(5), int64(1), object(7)
memory usage: 677.9+ MB


In [78]:

df_cleaned['ride_length_m'] = (df_cleaned['ended_at'] - df_cleaned['started_at']).dt.total_seconds() / 60

mean_ride = df_cleaned['ride_length_m'].mean()
max_ride = df_cleaned['ride_length_m'].max()
mode_day = df_cleaned['day_of_week'].mode()[0]

print(f"Mean Ride Length: {mean_ride:.2f} minutes")
print(f"Max Ride Length: {max_ride:.2f} minutes")
print(f"Most Common Day: {mode_day}")






Mean Ride Length: 16.03 minutes
Max Ride Length: 1574.90 minutes
Most Common Day: 7


In [81]:
summary_stats = df_cleaned.groupby(['member_casual', 'day_of_week'])['ride_length'].mean().reset_index()

#print(df_cleaned.groupby('member_casual')['ride_length_m'].mean())
#summary_stats.to_csv('cyclistic_summary.csv', index=False)
# Create a pivot table for the Case Study document
pivot_table = df_cleaned.groupby(['member_casual', 'day_of_week'])['ride_length_m'].mean().unstack()
print(pivot_table)

day_of_week            1          2          3          4          5  \
member_casual                                                          
casual         26.139672  22.331166  19.857857  18.715941  19.838213   
member         13.642907  11.885840  11.952587  11.784625  11.896964   

day_of_week            6          7  
member_casual                        
casual         22.505683  25.257423  
member         12.277501  13.483976  


In [82]:
import plotly.express as px

# 1. Prepare the data
# Mapping numbers to names for the x-axis
day_map = {1: 'Sun', 2: 'Mon', 3: 'Tue', 4: 'Wed', 5: 'Thu', 6: 'Fri', 7: 'Sat'}
plot_data = df_cleaned.groupby(['member_casual', 'day_of_week'])['ride_length_m'].mean().reset_index()
plot_data['day_name'] = plot_data['day_of_week'].map(day_map)

# 2. Create the Interactive Bar Chart
fig = px.bar(plot_data, 
             x='day_name', 
             y='ride_length_m', 
             color='member_casual',
             barmode='group',
             title='<b>Average Ride Duration: Casual vs. Member</b>',
             labels={'ride_length_m': 'Avg Minutes', 'day_name': 'Day of Week', 'member_casual': 'User Type'},
             color_discrete_map={'member': '#2E86AB', 'casual': '#F24236'}, # Your modern colors
             template='plotly_white')

# 3. Refine the layout for a "Premium" look
fig.update_layout(
    font_family="Arial",
    title_font_size=24,
    legend_title_text='User Type',
    hovermode="x unified" # Shows both member & casual values when hovering
)

# Show the plot
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed